In [ ]:
#!/usr/bin/env python
# coding: utf-8

# # Calcium Imaging Analysis Pipeline - Measurement 1 Only
# ## Analyzes only the first measurement files
# 
# This version:
# 1. For RNA experiment: Only processes files with "Meas1"
# 2. For Chemical experiment: Only processes files with "1001" 
# 3. No "Other Measurements" analysis
# 4. Peak detection: prominence > 0.2, minimum separation = 5 frames
# 5. Z-score outlier removal with threshold 3.0

# ## Import Libraries and Setup

import os
import re
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')  # Set backend before importing pyplot
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
import seaborn as sns
from scipy.signal import find_peaks, peak_widths
from scipy.stats import linregress, kruskal, mannwhitneyu, shapiro, sem, ttest_ind, zscore
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Setup plotting style
sns.set_style("whitegrid")
sns.set_context("paper", font_scale=1.2)

plt.rcParams.update({
    'font.family': 'sans-serif',
    'font.size': 11,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'figure.dpi': 100,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
    'axes.spines.top': False,
    'axes.spines.right': False,
    'pdf.fonttype': 42,
    'axes.grid': True
})

print("Setup complete!")
print(f"Analysis date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\nAnalysis Parameters:")
print("- Peak detection: prominence > 0.2, minimum separation = 5 frames")
print("- Active cells: ≥ 2 peaks")
print("- Outlier removal: z-score method with threshold = 3.0")
print("- Statistics: Shapiro-Wilk test → Mann-Whitney U or t-test")
print("- Analyzing ONLY Measurement 1 files\n")


# ## Define Core Functions

def parse_filename(filename, experiment_type='RNA'):
    """
    Parse filename to extract condition and measurement.
    Only accepts Measurement 1 files.
    """
    if experiment_type == 'RNA':
        # For RNA, only accept files with "Meas1"
        if 'Meas1' not in filename:
            return None, None, None
            
        filename_lower = filename.lower()
        
        # Determine condition for RNA
        if 'Scr' in filename or 'scr' in filename_lower or 'scrambled' in filename_lower:
            condition = 'Control (Scr)'
        elif 'siRNA' in filename or 'sirna' in filename_lower:
            condition = 'siRNA'
        elif 'saRNA' in filename or 'sarna' in filename_lower:
            condition = 'saRNA'
        else:
            return None, None, None
        
        # Check for dose patterns to exclude
        dose_patterns = ['µm', 'um', 'µl', 'ul']
        has_dose = any(pattern in filename_lower for pattern in dose_patterns)
        has_dose_number = any(num + 'µ' in filename or num + 'u' in filename_lower 
                             for num in ['100', '50', '25'])
        
        if has_dose and has_dose_number:
            return None, None, None  # Skip dose files
            
        return condition, 1, experiment_type
        
    else:  # Chemical experiment
        # For chemicals, only accept files with "1001"
        if '1001' not in filename:
            return None, None, None
            
        filename_lower = filename.lower()
        
        # Determine condition
        if 'wt' in filename_lower:
            condition = 'WT'
        elif 'nav' in filename_lower:
            condition = 'Nav1.1 activator'
        elif 'ptz' in filename_lower:
            condition = 'PTZ'
        elif 'vera' in filename_lower or 'veratridine' in filename_lower:
            condition = 'Veratridine'
        else:
            return None, None, None
    
        return condition, 1, experiment_type


def calculate_kinetics_peaks(file_path):
    """
    Calculate kinetics from CSV file with peak detection parameters.
    Peak detection uses prominence > 0.2 and minimum separation of 5 frames.
    Active cells are defined as having ≥ 2 peaks.
    """
    data = pd.read_csv(file_path, header=None, skiprows=1)
    frame_col, roi_cols = 0, data.columns[1:]
    
    results = []
    for roi_idx, roi_col in enumerate(roi_cols):
        frames = data[frame_col].dropna().values
        intensity = data[roi_col].dropna().values
        
        if len(frames) < 2 or len(intensity) < 2:
            continue
        
        baseline = np.mean(intensity)
        if baseline <= 0:
            continue
        
        dff = (intensity - baseline) / baseline
        if np.any(np.isnan(dff)) or np.any(np.isinf(dff)):
            continue
        
        # Find peaks with specified parameters
        peaks, properties = find_peaks(dff, prominence=0.2, distance=5, width=1)
        
        # Calculate metrics
        amplitude = np.mean(dff[peaks]) if peaks.size else 0
        slope = linregress(frames, dff).slope if peaks.size else 0
        widths, _, _, _ = peak_widths(dff, peaks, rel_height=0.5)
        fwhm_val = np.mean(widths) if widths.size else 0
        
        results.append({
            'NumPeaks': len(peaks),
            'Amplitude': amplitude,
            'Slope': slope,
            'FWHM': fwhm_val,
            'IsActive': len(peaks) >= 2,  # Active if 2 or more peaks
            'ROI_Index': roi_idx,
            'FilePath': file_path
        })
    
    return results


def remove_outliers_zscore(df, metric, threshold=3.0):
    """
    Remove outliers using z-score method with specified threshold.
    Default threshold is 3.0 as per methodology.
    """
    initial_count = len(df)
    
    # Calculate z-scores
    z_scores = np.abs(zscore(df[metric]))
    
    # Keep only data within threshold
    df_clean = df[z_scores < threshold].copy()
    
    removed = initial_count - len(df_clean)
    if removed > 0:
        print(f"Removed {removed} outliers from {metric} using z-score method (threshold={threshold}, {removed/initial_count*100:.1f}%)")
    
    return df_clean


def collect_measurement1_data(folder_path, experiment_type='RNA'):
    """Collect data from folder - ONLY Measurement 1 files."""
    data = []
    processed_files = 0
    skipped_files = 0
    parsing_log = []
    
    print(f"Looking for CSV files in: {folder_path}")
    print(f"Filtering for Measurement 1 only ({'Meas1' if experiment_type == 'RNA' else '1001'} files)")
    
    # List all files found
    all_csv_files = []
    
    # Use os.walk to handle both cases (files in root and in subfolders)
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv'):
                full_path = os.path.join(root, file)
                relative_path = os.path.relpath(full_path, folder_path)
                all_csv_files.append((file, full_path, relative_path))
    
    print(f"Found {len(all_csv_files)} CSV files total\n")
    
    # Print parsing header
    print("FILE PARSING DETAILS (Measurement 1 Only):")
    print("-" * 100)
    print(f"{'Filename':<50} {'Condition':<20} {'Measurement':<15} {'Status':<15}")
    print("-" * 100)
    
    # Track unique files to avoid duplicates
    processed_paths = set()
    
    # Process each CSV file
    for filename, full_path, rel_path in all_csv_files:
        # Skip if already processed (avoid duplicates)
        if full_path in processed_paths:
            continue
            
        # Check if it's a results file
        if 'results' in filename.lower() or experiment_type == 'Chemical':
            condition, measurement, _ = parse_filename(filename, experiment_type)
            
            if condition is None or measurement is None:
                skipped_files += 1
                status = "SKIPPED"
                reason = "Not Meas1" if experiment_type == 'RNA' and 'Meas' in filename and 'Meas1' not in filename else "No valid condition or dose file"
                parsing_log.append({
                    'filename': filename,
                    'condition': 'N/A',
                    'measurement': 'N/A',
                    'status': status,
                    'reason': reason
                })
                print(f"{filename:<50} {'N/A':<20} {'N/A':<15} {status:<15}")
            else:
                processed_files += 1
                processed_paths.add(full_path)
                status = "PROCESSED"
                parsing_log.append({
                    'filename': filename,
                    'condition': condition,
                    'measurement': str(measurement),
                    'status': status,
                    'reason': 'Success - Measurement 1'
                })
                
                try:
                    kinetics = calculate_kinetics_peaks(full_path)
                    
                    for k in kinetics:
                        k.update({
                            'Condition': condition,
                            'Measurement': measurement,
                            'File': filename,
                            'FilePath': full_path
                        })
                        data.append(k)
                    
                    print(f"{filename:<50} {condition:<20} {measurement:<15} {status:<15}")
                except Exception as e:
                    print(f"{filename:<50} {condition:<20} {measurement:<15} ERROR: {str(e)[:30]}")
                    status = "ERROR"
    
    print("-" * 100)
    print(f"\nSummary: Processed {processed_files} Measurement 1 files, skipped {skipped_files} files")
    print(f"Collected {len(data)} ROIs from Measurement 1\n")
    
    # Create parsing log DataFrame
    parsing_df = pd.DataFrame(parsing_log)
    
    return pd.DataFrame(data), parsing_df


# ## Calcium Trace Visualization Functions

def plot_calcium_traces(file_path, output_folder, max_traces_per_page=6):
    """
    Plot calcium traces for all ROIs in a file with marked peaks.
    Creates PDF files with multiple traces per page.
    """
    # Read the data
    data = pd.read_csv(file_path, header=None, skiprows=1)
    frame_col = 0
    roi_cols = data.columns[1:]
    
    # Create PDF filename based on input filename
    base_filename = os.path.splitext(os.path.basename(file_path))[0]
    pdf_filename = os.path.join(output_folder, f"{base_filename}_traces.pdf")
    
    # Calculate total pages needed
    total_rois = len(roi_cols)
    total_pages = (total_rois + max_traces_per_page - 1) // max_traces_per_page
    
    with matplotlib.backends.backend_pdf.PdfPages(pdf_filename) as pdf:
        roi_idx = 0
        
        for page in range(total_pages):
            # Create figure with subplots
            traces_on_page = min(max_traces_per_page, total_rois - roi_idx)
            fig, axes = plt.subplots(traces_on_page, 1, figsize=(12, 2.5 * traces_on_page))
            if traces_on_page == 1:
                axes = [axes]
            
            fig.suptitle(f'{base_filename} - Page {page + 1}/{total_pages}', fontsize=14, y=0.98)
            
            for i in range(traces_on_page):
                roi_col = roi_cols[roi_idx]
                ax = axes[i]
                
                # Get data
                frames = data[frame_col].dropna().values
                intensity = data[roi_col].dropna().values
                
                if len(frames) < 2 or len(intensity) < 2:
                    ax.text(0.5, 0.5, 'No data', ha='center', va='center', transform=ax.transAxes)
                    ax.set_xlabel('Frames')
                    ax.set_ylabel('ΔF/F')
                    roi_idx += 1
                    continue
                
                # Calculate ΔF/F
                baseline = np.mean(intensity)
                if baseline <= 0:
                    ax.text(0.5, 0.5, 'Invalid baseline', ha='center', va='center', transform=ax.transAxes)
                    ax.set_xlabel('Frames')
                    ax.set_ylabel('ΔF/F')
                    roi_idx += 1
                    continue
                
                dff = (intensity - baseline) / baseline
                
                # Plot the trace
                ax.plot(frames, dff, 'k-', linewidth=1, alpha=0.8)
                
                # Find and mark peaks with specified parameters
                peaks, properties = find_peaks(dff, prominence=0.2, distance=5, width=1)
                
                if len(peaks) > 0:
                    # Mark peaks with red dots
                    ax.plot(frames[peaks], dff[peaks], 'ro', markersize=6, label=f'{len(peaks)} peaks')
                    
                    # Add shaded regions around peaks
                    for peak_idx in peaks:
                        # Get peak width
                        widths, width_heights, left_ips, right_ips = peak_widths(
                            dff, [peak_idx], rel_height=0.5)
                        
                        if len(widths) > 0:
                            left_idx = int(left_ips[0])
                            right_idx = int(right_ips[0])
                            
                            # Add subtle shading
                            ax.axvspan(frames[left_idx], frames[right_idx], 
                                     alpha=0.2, color='red', zorder=0)
                
                # Formatting
                ax.set_xlabel('Frames', fontsize=10)
                ax.set_ylabel('ΔF/F', fontsize=10)
                ax.set_title(f'ROI {roi_idx + 1} - {"Active" if len(peaks) >= 2 else "Inactive"} ({len(peaks)} peaks)', fontsize=11, pad=5)
                ax.grid(True, alpha=0.3)
                ax.spines['top'].set_visible(False)
                ax.spines['right'].set_visible(False)
                
                # Add baseline reference
                ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5, linewidth=1)
                
                # Set y-limits with some padding
                y_min, y_max = ax.get_ylim()
                y_range = y_max - y_min
                ax.set_ylim(y_min - 0.1 * y_range, y_max + 0.1 * y_range)
                
                if len(peaks) > 0:
                    ax.legend(loc='upper right', fontsize=9, frameon=False)
                
                roi_idx += 1
            
            plt.tight_layout()
            pdf.savefig(fig, bbox_inches='tight')
            plt.close(fig)
    
    return pdf_filename


def create_trace_pdfs_measurement1(df, output_folder, experiment_type):
    """
    Create PDF files with calcium traces for all Measurement 1 files.
    """
    # Create traces subfolder
    traces_folder = output_folder
    os.makedirs(traces_folder, exist_ok=True)
    
    # Check if FilePath column exists
    if 'FilePath' not in df.columns:
        print("Warning: FilePath column not found in dataframe. Cannot create trace PDFs.")
        return []
    
    # Get unique files and their paths (all should be Measurement 1)
    unique_files = df[['File', 'FilePath']].drop_duplicates()
    
    print(f"\nCreating trace PDFs for all {len(unique_files)} Measurement 1 files")
    
    # Create subfolders by condition
    for condition in df['Condition'].unique():
        condition_folder = os.path.join(traces_folder, condition.replace(' ', '_').replace('(', '').replace(')', ''))
        os.makedirs(condition_folder, exist_ok=True)
    
    # Process each file
    pdf_files_created = []
    for idx, (_, row) in enumerate(unique_files.iterrows()):
        file_name = row['File']
        file_path = row['FilePath']
        
        # Get condition for this file
        condition = df[df['File'] == file_name]['Condition'].iloc[0]
        condition_folder = os.path.join(traces_folder, condition.replace(' ', '_').replace('(', '').replace(')', ''))
        
        print(f"  Processing traces for: {file_name} ({idx + 1}/{len(unique_files)})")
        
        try:
            pdf_path = plot_calcium_traces(file_path, condition_folder)
            pdf_files_created.append(pdf_path)
        except Exception as e:
            print(f"    Error creating traces for {file_name}: {str(e)}")
    
    print(f"\nCreated {len(pdf_files_created)} PDF files with calcium traces")
    
    # Create summary PDF with example traces from each condition
    create_trace_summary_pdf(df, traces_folder, experiment_type)
    
    return pdf_files_created


def create_trace_summary_pdf(df, traces_folder, experiment_type):
    """Create a summary PDF with example traces from each condition."""
    summary_pdf = os.path.join(traces_folder, f'{experiment_type}_Measurement1_trace_examples.pdf')
    
    conditions = sorted(df['Condition'].unique())
    
    with matplotlib.backends.backend_pdf.PdfPages(summary_pdf) as pdf:
        # Create overview page
        fig = plt.figure(figsize=(12, 9))
        fig.suptitle(f'{experiment_type} - Example Calcium Traces by Condition (Measurement 1)', fontsize=16)
        
        # Create grid of subplots
        n_conditions = len(conditions)
        n_cols = 2
        n_rows = (n_conditions + n_cols - 1) // n_cols
        
        for idx, condition in enumerate(conditions):
            ax = plt.subplot(n_rows, n_cols, idx + 1)
            
            # Get an example active ROI from this condition
            active_rois = df[(df['Condition'] == condition) & (df['IsActive'])]
            
            if len(active_rois) > 0:
                # Pick a representative ROI (one with median number of peaks)
                median_peaks = active_rois['NumPeaks'].median()
                example_roi = active_rois.iloc[(active_rois['NumPeaks'] - median_peaks).abs().argsort()[:1]]
                
                file_path = example_roi['FilePath'].iloc[0]
                roi_index = example_roi['ROI_Index'].iloc[0] if 'ROI_Index' in example_roi.columns else 1
                
                # Read and plot the trace
                try:
                    data = pd.read_csv(file_path, header=None, skiprows=1)
                    frames = data[0].dropna().values
                    
                    # Use the stored ROI index
                    if roi_index + 1 < len(data.columns):
                        intensity = data[roi_index + 1].dropna().values
                    else:
                        intensity = data[1].dropna().values  # Use first ROI as fallback
                    
                    baseline = np.mean(intensity)
                    if baseline > 0:
                        dff = (intensity - baseline) / baseline
                        
                        # Plot
                        ax.plot(frames, dff, 'k-', linewidth=1)
                        
                        # Find and mark peaks with specified parameters
                        peaks, _ = find_peaks(dff, prominence=0.2, distance=5, width=1)
                        if len(peaks) > 0:
                            ax.plot(frames[peaks], dff[peaks], 'ro', markersize=4)
                        
                        ax.set_title(f'{condition}\n({len(peaks)} peaks)', fontsize=12)
                        ax.set_xlabel('Frames', fontsize=10)
                        ax.set_ylabel('ΔF/F', fontsize=10)
                        ax.grid(True, alpha=0.3)
                        ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
                    else:
                        ax.text(0.5, 0.5, 'No valid example', ha='center', va='center', transform=ax.transAxes)
                except Exception as e:
                    ax.text(0.5, 0.5, f'Error: {str(e)[:20]}...', ha='center', va='center', transform=ax.transAxes)
            else:
                ax.text(0.5, 0.5, 'No active ROIs', ha='center', va='center', transform=ax.transAxes)
            
            ax.set_xlim(0, 100)  # Show first 100 frames
        
        plt.tight_layout()
        pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)
    
    print(f"Created summary PDF: {summary_pdf}")


# ## Statistical Functions

def perform_statistical_analysis(df, metric, conditions, control_condition):
    """
    Perform comprehensive statistical analysis comparing all conditions to control.
    Uses Shapiro-Wilk test for normality assessment, then chooses appropriate test:
    - Normal distribution: Independent t-test
    - Non-normal distribution or small samples: Mann-Whitney U test
    """
    results = {
        'metric': metric,
        'control': control_condition,
        'normality': {},
        'descriptive': {},
        'comparisons': []
    }
    
    # Descriptive statistics and normality test for each condition
    for cond in conditions:
        cond_data = df[df['Condition'] == cond][metric].dropna()
        if len(cond_data) >= 3:
            # Normality test (Shapiro-Wilk)
            stat, p_val = shapiro(cond_data)
            is_normal = p_val > 0.05
            
            # Descriptive stats
            results['descriptive'][cond] = {
                'n': len(cond_data),
                'mean': np.mean(cond_data),
                'median': np.median(cond_data),
                'std': np.std(cond_data),
                'sem': sem(cond_data),
                'min': np.min(cond_data),
                'max': np.max(cond_data)
            }
            results['normality'][cond] = {
                'is_normal': is_normal,
                'shapiro_p': p_val
            }
        else:
            results['descriptive'][cond] = {'n': len(cond_data)}
            results['normality'][cond] = {'is_normal': False, 'shapiro_p': np.nan}
    
    # Compare each condition to control
    control_data = df[df['Condition'] == control_condition][metric].dropna()
    if len(control_data) >= 2:
        for cond in conditions:
            if cond == control_condition:
                continue
                
            cond_data = df[df['Condition'] == cond][metric].dropna()
            if len(cond_data) >= 2:
                # Choose test based on normality
                both_normal = (results['normality'][control_condition]['is_normal'] and 
                             results['normality'][cond]['is_normal'])
                
                if both_normal and len(control_data) > 20 and len(cond_data) > 20:
                    # Use t-test for normal data with sufficient sample size
                    stat, p_val = ttest_ind(control_data, cond_data)
                    test_used = "Independent t-test"
                else:
                    # Use Mann-Whitney U for non-normal or small samples
                    stat, p_val = mannwhitneyu(control_data, cond_data, alternative='two-sided')
                    test_used = "Mann-Whitney U"
                
                # Calculate effect size (Cohen's d)
                pooled_std = np.sqrt((np.std(control_data)**2 + np.std(cond_data)**2) / 2)
                if pooled_std > 0:
                    cohens_d = (np.mean(cond_data) - np.mean(control_data)) / pooled_std
                else:
                    cohens_d = 0
                
                results['comparisons'].append({
                    'comparison': f"{cond} vs {control_condition}",
                    'test': test_used,
                    'statistic': stat,
                    'p_value': p_val,
                    'significant': p_val < 0.05,
                    'cohens_d': cohens_d,
                    'mean_diff': np.mean(cond_data) - np.mean(control_data)
                })
    
    return results


def generate_stats_report(df, output_path, experiment_name, experiment_type="RNA"):
    """Generate comprehensive statistical report with UTF-8 encoding."""
    # Determine control condition
    control_condition = 'Control (Scr)' if experiment_type == "RNA" else 'WT'
    
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(f"COMPREHENSIVE STATISTICAL ANALYSIS REPORT\n")
        f.write(f"{experiment_name}\n")
        f.write("="*80 + "\n\n")
        f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        # Analysis methodology
        f.write("ANALYSIS METHODOLOGY\n")
        f.write("-"*40 + "\n")
        f.write("Data Selection:\n")
        f.write("  - Only Measurement 1 data analyzed\n")
        f.write(f"  - RNA: Files with 'Meas1' in filename\n")
        f.write(f"  - Chemical: Files with '1001' in filename\n")
        f.write("Peak Detection Parameters:\n")
        f.write("  - Prominence threshold: > 0.2\n")
        f.write("  - Minimum peak separation: 5 frames\n")
        f.write("  - Active cells: ≥ 2 peaks\n")
        f.write("Outlier Removal:\n")
        f.write("  - Method: Z-score\n")
        f.write("  - Threshold: 3.0\n")
        f.write("Statistical Tests:\n")
        f.write("  - Normality assessment: Shapiro-Wilk test\n")
        f.write("  - Comparison tests: Mann-Whitney U (non-normal) or t-test (normal)\n\n")
        
        # Summary statistics
        f.write("OVERALL SUMMARY (Measurement 1 Only)\n")
        f.write("-"*40 + "\n")
        f.write(f"Total ROIs analyzed: {len(df)}\n")
        f.write(f"Active ROIs (≥2 peaks): {df['IsActive'].sum()} ({df['IsActive'].mean()*100:.1f}%)\n")
        f.write(f"Conditions: {sorted(df['Condition'].unique())}\n")
        f.write(f"Control condition: {control_condition}\n\n")
        
        # Activity by condition
        f.write("ACTIVITY BY CONDITION\n")
        f.write("-"*40 + "\n")
        f.write(f"{'Condition':<20} {'Total ROIs':<12} {'Active':<12} {'Percent':<12}\n")
        f.write("-"*60 + "\n")
        
        for cond in sorted(df['Condition'].unique()):
            cond_df = df[df['Condition'] == cond]
            active = cond_df['IsActive'].sum()
            total = len(cond_df)
            percent = active/total*100 if total > 0 else 0
            f.write(f"{cond:<20} {total:<12} {active:<12} {percent:<12.1f}%\n")
        
        # Analyze active cells only
        df_active = df[df['IsActive']].copy()
        
        if len(df_active) > 0:
            f.write("\n\nDETAILED STATISTICAL ANALYSIS (Active Cells Only)\n")
            f.write("="*80 + "\n\n")
            
            metrics = ['NumPeaks', 'Amplitude', 'Slope', 'FWHM']
            conditions = sorted(df_active['Condition'].unique())
            
            for metric in metrics:
                f.write(f"\n{metric.upper()}\n")
                f.write("-"*60 + "\n\n")
                
                # Run statistical analysis
                stats_results = perform_statistical_analysis(df_active, metric, conditions, control_condition)
                
                # Descriptive statistics table
                f.write("Descriptive Statistics:\n")
                f.write(f"{'Condition':<20} {'N':<6} {'Mean ± SEM':<15} {'Median':<10} {'Min-Max':<20}\n")
                f.write("-"*75 + "\n")
                
                for cond in conditions:
                    if cond in stats_results['descriptive']:
                        desc = stats_results['descriptive'][cond]
                        if desc['n'] > 0:
                            mean_sem = f"{desc['mean']:.3f} ± {desc['sem']:.3f}"
                            min_max = f"{desc['min']:.3f} - {desc['max']:.3f}"
                            f.write(f"{cond:<20} {desc['n']:<6} {mean_sem:<15} "
                                   f"{desc['median']:<10.3f} {min_max:<20}\n")
                
                # Normality test results
                f.write("\nNormality Test (Shapiro-Wilk):\n")
                for cond in conditions:
                    if cond in stats_results['normality']:
                        norm = stats_results['normality'][cond]
                        if not np.isnan(norm['shapiro_p']):
                            normal_str = "Normal" if norm['is_normal'] else "Not Normal"
                            f.write(f"{cond}: p = {norm['shapiro_p']:.4f} ({normal_str})\n")
                
                # Statistical comparisons vs control
                f.write(f"\nStatistical Comparisons (vs {control_condition}):\n")
                f.write("-"*60 + "\n")
                
                for comp in stats_results['comparisons']:
                    sig_str = "***" if comp['p_value'] < 0.001 else "**" if comp['p_value'] < 0.01 else "*" if comp['p_value'] < 0.05 else "ns"
                    effect_size = "Large" if abs(comp['cohens_d']) > 0.8 else "Medium" if abs(comp['cohens_d']) > 0.5 else "Small"
                    
                    f.write(f"{comp['comparison']}: {comp['test']}\n")
                    f.write(f"  p-value: {comp['p_value']:.4f} ({sig_str})\n")
                    f.write(f"  Mean difference: {comp['mean_diff']:.3f}\n")
                    f.write(f"  Effect size (Cohen's d): {comp['cohens_d']:.3f} ({effect_size})\n\n")
        
        # Footer
        f.write("\n" + "="*80 + "\n")
        f.write("STATISTICAL NOTATION:\n")
        f.write("ns: not significant (p > 0.05)\n")
        f.write("*: p < 0.05\n")
        f.write("**: p < 0.01\n")
        f.write("***: p < 0.001\n")
        f.write("\nEffect Size Interpretation (Cohen's d):\n")
        f.write("Small: |d| < 0.5\n")
        f.write("Medium: 0.5 ≤ |d| < 0.8\n")
        f.write("Large: |d| ≥ 0.8\n")


# ## Plotting Functions

def create_analysis_plots(df, metrics, output_folder, title_suffix="", experiment_type="RNA"):
    """Create analysis plots for active cells with updated violin plots and statistics."""
    df_active_original = df[df['IsActive']].copy()
    
    if len(df_active_original) == 0:
        print("No active cells found!")
        return
    
    # Create figure
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()
    
    # Define condition order and control based on experiment type
    if experiment_type == "RNA":
        condition_order = ['Control (Scr)', 'saRNA', 'siRNA']
        control_condition = 'Control (Scr)'
        color_map = {
            'Control (Scr)': '#1f77b4',  # Blue
            'saRNA': '#ff7f0e',          # Orange
            'siRNA': '#2ca02c'           # Green
        }
    else:
        condition_order = ['WT', 'Nav1.1 activator', 'PTZ', 'Veratridine']
        control_condition = 'WT'
        color_map = {
            'WT': '#1f77b4',
            'Nav1.1 activator': '#ff7f0e',
            'PTZ': '#2ca02c',
            'Veratridine': '#d62728'
        }
    
    # Filter to only include conditions present in data
    conditions = [c for c in condition_order if c in df_active_original['Condition'].unique()]
    colors = [color_map.get(c, '#666666') for c in conditions]
    
    for idx, metric in enumerate(metrics):
        ax = axes[idx]
        
        # Remove outliers for THIS metric only
        df_active = remove_outliers_zscore(df_active_original.copy(), metric, threshold=3.0)
        
        # Prepare data in correct order
        data_to_plot = []
        positions = []
        for i, cond in enumerate(conditions):
            cond_data = df_active[df_active['Condition'] == cond][metric].values
            if len(cond_data) > 0:
                data_to_plot.append(cond_data)
                positions.append(i + 1)
        
        # Skip if no data to plot
        if not data_to_plot:
            continue
            
        # Create violin plot with error handling
        try:
            parts = ax.violinplot(data_to_plot, positions=positions, 
                                 showmeans=False, showmedians=False, showextrema=False)
            
            # Customize violins
            for i, pc in enumerate(parts['bodies']):
                pc.set_facecolor(colors[i])
                pc.set_alpha(0.7)
                pc.set_edgecolor('black')
                pc.set_linewidth(1)
        except Exception as e:
            print(f"Warning: Could not create violin plot for {metric}: {str(e)}")
            # Fallback to box plot
            bp = ax.boxplot(data_to_plot, positions=positions, patch_artist=True)
            for i, box in enumerate(bp['boxes']):
                box.set_facecolor(colors[i])
                box.set_alpha(0.7)
        
        # Add only mean lines
        for i, (pos, data) in enumerate(zip(positions, data_to_plot)):
            if len(data) > 0:
                mean_val = np.mean(data)
                ax.hlines(mean_val, pos-0.2, pos+0.2, colors='red', linewidth=3)
        
        # Run statistical analysis
        stats_results = perform_statistical_analysis(df_active, metric, conditions, control_condition)
        
        # Add significance annotations
        y_max = ax.get_ylim()[1]
        y_range = ax.get_ylim()[1] - ax.get_ylim()[0]
        annotation_height = y_max + 0.05 * y_range
        
        for comp in stats_results['comparisons']:
            # Find positions for the comparison
            comp_parts = comp['comparison'].split(' vs ')
            if comp_parts[0] in conditions and comp_parts[1] in conditions:
                idx1 = conditions.index(comp_parts[0])
                idx2 = conditions.index(comp_parts[1])
                
                if comp['significant']:
                    sig_str = "***" if comp['p_value'] < 0.001 else "**" if comp['p_value'] < 0.01 else "*"
                    
                    # Draw bracket
                    x1, x2 = positions[idx1], positions[idx2]
                    ax.plot([x1, x1, x2, x2], 
                           [annotation_height - 0.02*y_range, annotation_height, 
                            annotation_height, annotation_height - 0.02*y_range], 
                           'k-', linewidth=1)
                    ax.text((x1 + x2) / 2, annotation_height + 0.01*y_range, sig_str, 
                           ha='center', va='bottom', fontsize=12)
                    annotation_height += 0.08 * y_range
        
        # Labels and formatting
        ax.set_xticks(positions)
        ax.set_xticklabels(conditions, rotation=45, ha='right')
        ax.set_ylabel(metric)
        ax.set_title(metric)
        ax.grid(True, alpha=0.3, axis='y')
        
        # Add sample sizes
        y_min = ax.get_ylim()[0]
        for i, (pos, cond) in enumerate(zip(positions, conditions)):
            n = len(df_active[df_active['Condition'] == cond][metric])
            ax.text(pos, y_min - 0.05 * y_range, 
                   f'n={n}', ha='center', fontsize=10, weight='bold')
        
        # Adjust y-axis to make room for n= labels and significance annotations
        ax.set_ylim(bottom=y_min - 0.1 * y_range, 
                   top=max(ax.get_ylim()[1], annotation_height + 0.05 * y_range))
    
    plt.suptitle(f'Calcium Imaging Analysis - Measurement 1{title_suffix}', fontsize=16)
    plt.tight_layout()
    
    # Save as both PNG and PDF with error handling
    try:
        plt.savefig(os.path.join(output_folder, f'analysis_measurement1{title_suffix.replace(" ", "_")}.png'), 
                    dpi=300, bbox_inches='tight')
        plt.savefig(os.path.join(output_folder, f'analysis_measurement1{title_suffix.replace(" ", "_")}.pdf'), 
                    format='pdf', bbox_inches='tight')
    except Exception as e:
        print(f"Warning: Could not save plots: {str(e)}")
    finally:
        plt.close()


def create_activity_plot(df, output_folder, title_suffix="", experiment_type="RNA"):
    """Create activity summary plot with individual file dots and SD error bars."""
    fig, ax = plt.subplots(1, 1, figsize=(10, 7))
    
    # Define condition order based on experiment type
    if experiment_type == "RNA":
        condition_order = ['Control (Scr)', 'saRNA', 'siRNA']
        control_condition = 'Control (Scr)'
        color_map = {
            'Control (Scr)': '#1f77b4',  # Blue
            'saRNA': '#ff7f0e',          # Orange
            'siRNA': '#2ca02c'           # Green
        }
    else:
        condition_order = ['WT', 'Nav1.1 activator', 'PTZ', 'Veratridine']
        control_condition = 'WT'
        color_map = {
            'WT': '#1f77b4',
            'Nav1.1 activator': '#ff7f0e',
            'PTZ': '#2ca02c',
            'Veratridine': '#d62728'
        }
    
    # Filter to only include conditions present in data
    conditions = [c for c in condition_order if c in df['Condition'].unique()]
    
    # Calculate activity by condition and FILE
    activity_by_file = []
    
    for cond in conditions:
        cond_df = df[df['Condition'] == cond]
        
        # Get unique files for this condition
        unique_files = cond_df['File'].unique()
        
        for file in unique_files:
            file_df = cond_df[cond_df['File'] == file]
            total = len(file_df)
            active = len(file_df[file_df['IsActive']])
            if total > 0:
                activity_by_file.append({
                    'Condition': cond,
                    'File': file,
                    'Active': active,
                    'Total': total,
                    'Percent': active/total*100
                })
    
    activity_detail_df = pd.DataFrame(activity_by_file)
    
    # Calculate overall statistics by condition
    activity_summary = []
    for cond in conditions:
        cond_data = activity_detail_df[activity_detail_df['Condition'] == cond]['Percent']
        if len(cond_data) > 0:
            mean_percent = cond_data.mean()
            std_percent = cond_data.std()
            sem_percent = cond_data.sem()
            total_cells = df[df['Condition'] == cond].shape[0]
            activity_summary.append({
                'Condition': cond,
                'MeanPercent': mean_percent,
                'StdPercent': std_percent,
                'SemPercent': sem_percent,
                'TotalCells': total_cells
            })
    
    activity_summary_df = pd.DataFrame(activity_summary)
    
    # Create bar plot with SD error bars
    x_positions = np.arange(len(conditions))
    colors = [color_map.get(c, '#666666') for c in conditions]
    
    # Plot bars with error bars
    bars = ax.bar(x_positions, activity_summary_df['MeanPercent'], 
                  yerr=activity_summary_df['StdPercent'],
                  color=colors, alpha=0.6, edgecolor='black', linewidth=1,
                  capsize=5, error_kw={'linewidth': 2})
    
    # Add individual file dots
    np.random.seed(42)  # For reproducible jitter
    for i, cond in enumerate(conditions):
        cond_data = activity_detail_df[activity_detail_df['Condition'] == cond]
        if not cond_data.empty:
            # Add some jitter to x positions for better visibility
            jitter = np.random.normal(0, 0.08, len(cond_data))
            x_dots = np.full(len(cond_data), i) + jitter
            
            # Plot dots for each file
            ax.scatter(x_dots, cond_data['Percent'], 
                      color='black', s=80, alpha=0.7, zorder=10,
                      edgecolors='white', linewidth=1.5)
    
    # Run statistical analysis for activity percentages
    if len(activity_detail_df) > 0:
        # Prepare data for statistical analysis
        stats_df = pd.DataFrame()
        for _, row in activity_detail_df.iterrows():
            stats_df = pd.concat([stats_df, pd.DataFrame({
                'Condition': [row['Condition']],
                'ActivityPercent': [row['Percent']]
            })], ignore_index=True)
        
        stats_results = perform_statistical_analysis(stats_df, 'ActivityPercent', 
                                                   conditions, control_condition)
        
        # Add significance annotations
        y_max = max(activity_detail_df['Percent'].max() * 1.1 if not activity_detail_df.empty else 50, 
                   (activity_summary_df['MeanPercent'] + activity_summary_df['StdPercent']).max() * 1.1 if not activity_summary_df.empty else 50)
        annotation_height = y_max
        
        if 'comparisons' in stats_results:
            for comp in stats_results['comparisons']:
                if comp['significant']:
                    # Find positions for the comparison
                    comp_parts = comp['comparison'].split(' vs ')
                    if comp_parts[0] in conditions and comp_parts[1] in conditions:
                        idx1 = conditions.index(comp_parts[0])
                        idx2 = conditions.index(comp_parts[1])
                        
                        sig_str = "***" if comp['p_value'] < 0.001 else "**" if comp['p_value'] < 0.01 else "*"
                        
                        # Draw bracket
                        bracket_height = annotation_height
                        ax.plot([idx1, idx1, idx2, idx2], 
                               [bracket_height - 1, bracket_height, bracket_height, bracket_height - 1], 
                               'k-', linewidth=1)
                        ax.text((idx1 + idx2) / 2, bracket_height + 0.5, sig_str, 
                               ha='center', va='bottom', fontsize=14, weight='bold')
                        annotation_height += 5
    
    # Labels and formatting
    ax.set_xticks(x_positions)
    ax.set_xticklabels(conditions, rotation=45, ha='right', fontsize=12)
    ax.set_ylabel('Active Cells (%)', fontsize=14)
    ax.set_title(f'Cell Activity by Condition - Measurement 1{title_suffix}\n(dots show individual files, bars show mean ± SD)', 
                fontsize=14)
    ax.set_ylim(0, max(annotation_height + 2, y_max))
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add text annotations on bars
    for i, (bar, row) in enumerate(zip(bars, activity_summary_df.itertuples())):
        height = row.MeanPercent
        ax.text(bar.get_x() + bar.get_width()/2., height + row.StdPercent + 1,
               f'{row.MeanPercent:.1f}%\n(n={row.TotalCells})',
               ha='center', va='bottom', fontsize=10)
    
    # Add legend if significant differences found
    if 'comparisons' in stats_results and any(comp['significant'] for comp in stats_results['comparisons']):
        ax.text(0.02, 0.98, 'Statistical significance vs ' + control_condition, 
               transform=ax.transAxes, va='top', ha='left', fontsize=10,
               bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    
    # Save as both PNG and PDF
    plt.savefig(os.path.join(output_folder, f'activity_measurement1{title_suffix.replace(" ", "_")}.png'), 
                dpi=300, bbox_inches='tight')
    plt.savefig(os.path.join(output_folder, f'activity_measurement1{title_suffix.replace(" ", "_")}.pdf'), 
                format='pdf', bbox_inches='tight')
    plt.close()


# ## Main Analysis Pipeline

def analyze_experiment_measurement1(base_path, experiment_name, experiment_type, output_base):
    """Run complete analysis for an experiment - Measurement 1 only."""
    print(f"\n{'='*60}")
    print(f"Analyzing: {experiment_name} - MEASUREMENT 1 ONLY")
    print(f"Path: {base_path}")
    print("="*60)
    
    # Create output directories
    exp_output = os.path.join(output_base, experiment_name.replace(' ', '_'))
    subdirs = {
        'figures': os.path.join(exp_output, 'Figures'),
        'stats': os.path.join(exp_output, 'Statistics'),
        'data': os.path.join(exp_output, 'Data'),
        'traces': os.path.join(exp_output, 'Calcium_Traces')
    }
    
    for dir_path in subdirs.values():
        os.makedirs(dir_path, exist_ok=True)
    
    # Collect data - Measurement 1 only
    print("\nCollecting Measurement 1 data...")
    df, parsing_log = collect_measurement1_data(base_path, experiment_type)
    
    if df.empty:
        print("No Measurement 1 data found!")
        return
    
    # Save parsing log
    parsing_log.to_csv(os.path.join(subdirs['data'], 'file_parsing_log_measurement1.csv'), index=False)
    
    print(f"\nMeasurement 1 Summary:")
    print(f"  Total ROIs: {len(df)}")
    print(f"  Active ROIs: {df['IsActive'].sum()} ({df['IsActive'].mean()*100:.1f}%)")
    print(f"  Conditions: {sorted(df['Condition'].unique())}")
    
    # Save raw data
    df.to_csv(os.path.join(subdirs['data'], 'measurement1_data.csv'), index=False)
    
    # Define metrics
    metrics = ['NumPeaks', 'Amplitude', 'Slope', 'FWHM']
    
    # Create analysis plots
    print(f"\nCreating analysis plots...")
    create_analysis_plots(df, metrics, subdirs['figures'], "", experiment_type)
    create_activity_plot(df, subdirs['figures'], "", experiment_type)
    
    # Generate statistics report
    print(f"\nGenerating statistical report...")
    generate_stats_report(df, os.path.join(subdirs['stats'], 'measurement_1_stats.txt'), 
                        f"{experiment_name} - Measurement 1", experiment_type)
    
    # Create calcium trace PDFs
    print("\nGenerating calcium trace PDFs...")
    create_trace_pdfs_measurement1(df, subdirs['traces'], experiment_type)
    
    print(f"\nAnalysis complete! Results saved to: {exp_output}")
    print(f"  - Parsing log: Data/file_parsing_log_measurement1.csv")
    print(f"  - Calcium traces: Calcium_Traces/")
    print(f"  - Statistics: Statistics/measurement_1_stats.txt")
    print(f"  - Figures: Figures/")


# ## Run Analysis

# Create timestamp for output
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_base = f"CalciumImaging_Measurement1_Results_{timestamp}"
os.makedirs(output_base, exist_ok=True)

print(f"Output directory: {output_base}\n")

# RNA Interference Experiment
rna_path = r'H:\Scripts June PhD\Calcium imaging June\Calcium imaging Scr, saRNA and siRNA'
if os.path.exists(rna_path):
    analyze_experiment_measurement1(rna_path, "RNA_Interference", "RNA", output_base)
else:
    print(f"RNA path not found: {rna_path}")

# Chemical Treatments Experiment  
chem_path = r'H:\Scripts June PhD\Calcium imaging June\20250303 Chemicals Results'
if os.path.exists(chem_path):
    analyze_experiment_measurement1(chem_path, "Chemical_Treatments", "Chemical", output_base)
else:
    print(f"Chemical path not found: {chem_path}")

print(f"\n{'='*60}")
print("ALL ANALYSES COMPLETE!")
print(f"Results saved to: {output_base}")
print(f"Figures saved as both PNG and PDF formats")
print(f"Calcium trace PDFs created for all Measurement 1 files")
print("="*60)


# ## Summary
# 
# Key changes in this version:
# 1. **Only analyzes Measurement 1 files**:
#    - RNA: Only files with "Meas1" in filename
#    - Chemical: Only files with "1001" in filename (not "2001")
# 2. **Removed duplicate processing** - tracks processed files
# 3. **Creates trace PDFs for ALL Measurement 1 files** (not just a sample)
# 4. **Single analysis output** - no separate "Other Measurements" folder
# 5. **Maintains all methodology parameters**:
#    - Peak detection: prominence > 0.2, separation = 5 frames
#    - Active cells: ≥ 2 peaks  
#    - Z-score outlier removal with threshold 3.0
#    - Shapiro-Wilk → Mann-Whitney U or t-test